# Testing evaluation Video Making

Generating an overlay frame for visual inspection

In [ ]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import pandas as pd
import cv2

import torch
import torch.nn.functional as F

import os

In [ ]:
path_to_data = "/share/data_ultraschall/compressions"

In [ ]:
# load the quality overview
quality = pd.read_csv("~/quality.csv")

# More info about the sequences
sequences = pd.read_csv("~/sequences.csv")

In [ ]:
quality

In [ ]:
sequences

In [ ]:
# filter based on the table above
useable = quality[
    (quality["Vessel in Frame"] <3) &
    (quality['LM configuration'] < 4) &
    (quality['Vein contrast/cropping error'] < 3) &
    (quality['Artery contrast'] < 3) &
    (quality['Vein boundary'] < 3) &
    (quality['Artery boundary'] < 3) &
    (quality['Gain'] == 1) &
    (quality['Artefacts'] < 3) &
    (quality['Movement'] < 3) &
    (quality['Total'] < 21)
]
useable

# filter based on useable
overview = sequences[sequences.Id.isin(useable.index.tolist())]
overview

In [ ]:
overview_clean = overview[overview['Id'].astype(str).isin(os.listdir(path_to_data))]
overview_clean

In [ ]:
available_seg = []
for proband in os.listdir(path_to_data):
    proband_path = os.path.join(path_to_data,str(proband))
    if "segmentations" in os.listdir(proband_path):
        available_seg.append(proband)
len(available_seg)

In [ ]:
for proband in available_seg:
    available_frames = len(os.listdir(os.path.join(path_to_data,proband,'frames')))
    if available_frames > 200:
        print(f"There are {available_frames} frames available for id {proband}")

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.animation as animation

import torch
import torch.nn
import torch.nn.functional as F

from PIL import Image

In [ ]:
prob_id='1164'
frame_path = os.path.join(path_to_data,prob_id,'frames')
seg_path = os.path.join(path_to_data,prob_id,'segmentations','1')

In [ ]:
frame_list = []
for frame in os.listdir(frame_path):
    frame_list.append(os.path.join(frame_path,frame))
frame_list.sort()

In [ ]:
seg_list = []
for seg in os.listdir(seg_path):
    seg_list.append(os.path.join(seg_path,seg))
seg_list.sort()

In [ ]:
print(f"{len(seg_list)} Segmentations and {len(frame_list)} Frames")

In [ ]:
# check if the overlay works
def overlaySegment(gray1,seg1,flag=False):
    H, W = seg1.squeeze().size()
    colors=torch.FloatTensor([0,0,0,199,67,66,225,140,154,78,129,170,45,170,170,240,110,38,111,163,91,235,175,86,202,255,52,162,0,183]).view(-1,3)/255.0
    segs1 = labelMatrixOneHot(seg1.unsqueeze(0),8)

    seg_color = torch.mm(segs1.view(8,-1).t(),colors[:8,:]).view(H,W,3)
    alpha = torch.clamp(1.0 - 0.5*(seg1>0).float(),0,1.0)

    overlay = (gray1*alpha).unsqueeze(2) + seg_color*(1.0-alpha).unsqueeze(2)
    if(flag):
        plt.imshow((overlay).numpy())
        plt.show()
    return overlay
def labelMatrixOneHot(segmentation, label_num):
    B, H, W = segmentation.size()
    values = segmentation.view(B,1,H,W).expand(B,label_num,H,W).to(segmentation.device)
    linspace = torch.linspace(0, label_num-1, label_num).long().view(1,label_num,1,1).expand(B,label_num,H,W).to(segmentation.device)
    matrix = (values.float()==linspace.float()).float().to(segmentation.device)
    for j in range(2,matrix.shape[1]):
        matrix[0,j,:,:] = matrix[0,j,:,:]
    return matrix

In [ ]:
%matplotlib notebook
assert len(frame_list) == len(seg_list)

frs = []
fig = plt.figure()
for i in range(len(frame_list)):
    loaded_frame = torch.from_numpy(np.array(Image.open(frame_list[i]))) / 255
    loaded_seg = torch.from_numpy(np.array(Image.open(seg_list[i]))) / 100

    overlay = overlaySegment(loaded_frame, loaded_seg, False)
    frs.append([plt.imshow(overlay, animated=True)])

ani = animation.ArtistAnimation(fig, frs, interval=50, blit=True, repeat_delay=1000)
plt.show()